# Model Implementation

## Initialize Packages

In [21]:
import sys
import pandas as pd
import numpy as np
import os
import rich
import xgboost as xgb
import json
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

## Load Data and Assign Predictors

In [3]:
model_data = pd.read_csv('../backend/model_data.csv')
inference_data = pd.read_csv('../backend/inference_data.csv')

In [4]:
# Define the list of predictors
# Create predictor list
veh_pred_lst = ['veh_value', 'veh_body', 'veh_age', 'engine_type', 'max_power', 'veh_color']
policy_pred_lst = ['gender', 'agecat', 'e_bill' ]
driving_behavior_pred_lst = ['area', 'time_of_week_driven', 'time_driven']
demo_pred_lst = ['marital_status', 'low_education_ind', 'credit_score', 'driving_history_score']
pred_lst = veh_pred_lst + policy_pred_lst + driving_behavior_pred_lst + demo_pred_lst # Split the data into training and validation sets
# pred_lst = ['engine_type', 'gender', 'credit_score', 'veh_age', 'agecat', 'area']

train_data = model_data.loc[model_data['sample'] == '1|bld']
val_data = model_data.loc[model_data['sample'] == '2|val']
rich.print( train_data.shape, val_data.shape )
train_data.head(5)

(11204, 25)
(3796, 25)

,id,fold,sample,veh_value,exposure,veh_body,veh_age,gender,area,agecat,...,e_bill,time_of_week_driven,time_driven,trm_len,credit_score,low_education_ind,clm,numclaims,claimcst0,expected_loss
0,1,2,1|bld,5.80,0.362191,SUV,2,F,B,1,...,0,weekday,12pm - 6pm,6,646.516469,0.0,1,1,2023.198184,5585.998969
1,2,3,1|bld,5.67,0.632068,STNWG,4,F,A,2,...,0,weekend,6am - 12pm,12,635.400369,0.0,1,1,3600.172234,5695.859540
2,3,1,1|bld,5.90,0.367460,SEDAN,2,F,C,2,...,0,weekday,6am - 12pm,12,646.463131,0.0,1,1,2021.144067,5500.307127
3,4,2,1|bld,4.79,0.802184,STNWG,3,M,B,4,...,1,weekday,6pm - 12am,12,645.598794,0.0,1,1,4006.845492,4994.920513
4,5,2,1|bld,6.68,0.485009,SEDAN,3,M,C,1,...,0,weekday,6am - 12pm,12,657.348612,0.0,1,1,2542.953931,5243.106060


In [5]:
# Define features and target
target = 'expected_loss'

X_train = train_data[pred_lst].copy()
y_train = train_data[target]

# Random Forest

In [13]:
for col in X_train.select_dtypes("object"):
    X_train[col] = X_train[col].astype("category").cat.codes

In [ ]:
# Train Random Forest
rf = RandomForestRegressor(
    n_estimators=300,      # number of trees
    max_depth=None,        # no depth limit (fully grown trees)
    random_state=42,
    n_jobs=1
)

rf.fit(X_train, y_train)

# In-sample predictions
y_pred = rf.predict(X_train)

# Evaluate on training data
rmse = root_mean_squared_error(y_train, y_pred)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

print("Training Model Metrics")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.3f}")

Training Model Metrics
RMSE: 1479.17
MAE: 671.65
R²: 0.859


# XGBoost

## Frequency-Severity Modeling

## Risk Segmentation

### Quantiles

In [19]:
train_data['predicted_expected_loss'] = y_pred

train_data.head()

/var/folders/nj/nbmj9gh56g9b50kd52cy7mwr0000gn/T/ipykernel_8963/2966020220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['predicted_expected_loss'] = y_pred


,id,fold,sample,veh_value,exposure,veh_body,veh_age,gender,area,agecat,...,time_of_week_driven,time_driven,trm_len,credit_score,low_education_ind,clm,numclaims,claimcst0,expected_loss,predicted_expected_loss
0,1,2,1|bld,5.80,0.362191,SUV,2,F,B,1,...,weekday,12pm - 6pm,6,646.516469,0.0,1,1,2023.198184,5585.998969,4190.744101
1,2,3,1|bld,5.67,0.632068,STNWG,4,F,A,2,...,weekend,6am - 12pm,12,635.400369,0.0,1,1,3600.172234,5695.859540,5273.055427
2,3,1,1|bld,5.90,0.367460,SEDAN,2,F,C,2,...,weekday,6am - 12pm,12,646.463131,0.0,1,1,2021.144067,5500.307127,4220.612971
3,4,2,1|bld,4.79,0.802184,STNWG,3,M,B,4,...,weekday,6pm - 12am,12,645.598794,0.0,1,1,4006.845492,4994.920513,3422.488947
4,5,2,1|bld,6.68,0.485009,SEDAN,3,M,C,1,...,weekday,6am - 12pm,12,657.348612,0.0,1,1,2542.953931,5243.106060,3485.267952


In [20]:
train_data["expected_loss"].quantile([0.33, 0.66])

0.33    0.0
0.66    0.0
Name: expected_loss, dtype: float64

### Fixed Thresholds

### KMeans Clustering

## Save Altered Data

In [ ]:
model_data.to_csv('../project/model_data.csv', index=False)
inference_data.to_csv('../project/inference_data.csv', index=False)

## Export Model Artifacts

In [24]:
# 1) Save the trained RandomForestRegressor
joblib.dump(rf, "../project/models/rf_model.pkl")

# 2) Save the exact feature column order used to fit the model
feature_columns = list(X_train.columns)  # if you trained with a DataFrame
with open("../project/models/feature_columns.json", "w") as f:
    json.dump(feature_columns, f)

# 3) (Optional) Create simple risk cutoffs from training predictions